# Génération du dataset pour l'analyse de données

Les données issues du scraping de l'API IDF Mobilités ne constituent pas un jeu de données en tant que tel. Le rôle de ce notebook est de passer du des données brutes à un jeu de données exploitable par la suite.

In [ ]:
import os
import json
import pandas as pd
import numpy as np

Pour avoir plus de flexibilité pour mener notre analyse, nous ne travaillerons pas seulement avec un dataset combinant les données sur les perturbations et les lignes affectées, nous garderons aussi de côté les données propres aux perturbations et aux lignes séparées.

In [ ]:
from scraping.config import fs, ROOT

paths = fs.ls(ROOT)

all_results = []
all_disruptions = []
all_lines = []

for file_path in paths:
    with open(file_path, 'r', encoding='ascii') as f:
        raw_data = f.read()
        data = json.loads(raw_data)
        
        last_updated = data.get('lastUpdatedDate')
        disruptions = data.get('disruptions', [])
        lines = data.get('lines', [])
        
        all_results.append({
            'lastUpdatedDate': last_updated,
            'disruptions': disruptions,
            'lines': lines
        })
        
        for d in disruptions:
            all_disruptions.append({
                'disruption_id': d.get('id'),
                'applicationPeriods': d.get('applicationPeriods'),
                'lastUpdate': d.get('lastUpdate'),
                'cause': d.get('cause'),
                'severity': d.get('severity'),
                'title': d.get('title'),
                'message': d.get('message'),
                'file_lastUpdatedDate': last_updated
            })
        
        for l in lines:
            all_lines.append({
                'line_id': l.get('id'),
                'name': l.get('name'),
                'shortName': l.get('shortName'),
                'mode': l.get('mode'),
                'networkId': l.get('networkId'),
                'impactedObjects': l.get('impactedObjects', []),
                'file_lastUpdatedDate': last_updated
            })

Du fait de perturbations pouvant être de longue durée (travaux...), nos données brutes comportent de nombreux duplicats que nous allons devoir traiter.

In [ ]:
df_results = pd.DataFrame(all_results)
df_disruptions = pd.DataFrame(all_disruptions)
df_lines = pd.DataFrame(all_lines)

print("Total disruptions (before):", len(df_disruptions))
print("Total lines (before):", len(df_lines))
print("Total results (before):", len(df_results))

df_results = df_results.drop_duplicates(subset=['lastUpdatedDate'])
df_disruptions = df_disruptions.drop_duplicates(subset=['disruption_id'])
df_lines = df_lines.drop_duplicates(subset=['line_id'])

print("Total disruptions (after):", len(df_disruptions))
print("Total lines (after):", len(df_lines))
print("Total results (after):", len(df_results))

In [ ]:
# jointure entre les lignes et les perturbations

line_disruption_links = []

for idx, row in df_lines.iterrows():
    impacted_objects = row['impactedObjects']
    line_id = row['line_id']
    if impacted_objects:
        for obj in impacted_objects:
            disruptions_ids = obj.get('disruptionsIds', [])
            for d_id in disruptions_ids:
                line_disruption_links.append({
                    'line_id': line_id,
                    'disruption_id': d_id
                })

df_line_disruption = pd.DataFrame(line_disruption_links)

# df_line_disruption.merge(df_lines, on='line_id') \
#                   .merge(df_disruptions, left_on='disruption_id', right_on='disruption_id')